# import

In [1]:
import sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


# Constants

In [2]:
BUCKET = "amazon-sagemaker-ai-by-example"
OUTPUT_PATH = f"s3://{BUCKET}/output"
TEST_PATH = f"s3://{BUCKET}/data/test"
TRAIN_PATH = f"s3://{BUCKET}/data/train"
VALIDATION_PATH = f"s3://{BUCKET}/data/validation"

# Setup

In [3]:
sess = sagemaker.Session()
region = sess.boto_region_name
container = sagemaker.image_uris.retrieve(region=region, framework="linear-learner")
role = sagemaker.get_execution_role()
linear = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.c4.xlarge",
    output_path=OUTPUT_PATH,
    sagemaker_session=sess,
)
linear.set_hyperparameters(predictor_type="regressor")
train_input = sagemaker.inputs.TrainingInput(
    TRAIN_PATH,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
    record_wrapping=None,
    compression=None,
)
validation_input = sagemaker.inputs.TrainingInput(
    VALIDATION_PATH,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
    record_wrapping=None,
    compression=None,
)
test_input = sagemaker.inputs.TrainingInput(
    TEST_PATH,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
    record_wrapping=None,
    compression=None,
)

# Train

In [4]:
%%time
linear.fit(inputs={
    "train": train_input,
    "validation": validation_input,
    "test": test_input
})

INFO:sagemaker:Creating training-job with name: linear-learner-2025-06-04-00-43-28-938


2025-06-04 00:43:31 Starting - Starting the training job...
2025-06-04 00:43:44 Starting - Preparing the instances for training...
2025-06-04 00:44:10 Downloading - Downloading input data...
2025-06-04 00:44:36 Downloading - Downloading the training image.........
2025-06-04 00:46:27 Training - Training image download completed. Training in progress..Docker entrypoint called with argument(s): train
Running default environment configuration script
[06/04/2025 00:46:34 INFO 140439620142912] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss'

# Model Data

In [5]:
linear.model_data

's3://amazon-sagemaker-ai-by-example/output/linear-learner-2025-06-04-00-43-28-938/output/model.tar.gz'